# Modeling Microbial Growth
## NCSSM Mini-term Feb. 2018
### Prepared by Cullen J.N. Roth | Magwene Lab 

# 1. Load modules and write functions for analysis of growth

## 1.1 Import needed modules

In [ ]:
import numpy as np ## Numpy is a useful family of functions in python for basic analysis
import pandas as pd ## Pandas is a groupof functions for loading in files and manipulate dataframes
from matplotlib import pyplot as plt ## Matplotib allows us to make pretty figures
from scipy.optimize import curve_fit ## The curve fit ftn allows us to fit curves

###### This line of code below allows us to plot within the notebook

In [ ]:
%matplotlib inline

## 1.2 Functions for analysis of microbial growth curves.
### Here we will define the growth models used in our analysis. These were taken from Zwietering et al 1990 (see table 2). Below we have defined the functions in terms of the growing capacity (A), the maximum growth rate (u) and the lag time (l). 

In [ ]:
def logistic(t,A,u,l):
    """Logistic growth equation"""
    y = A/(1+np.exp(((4*u/A)*(l - t)) + 2))
    return y

def gompertz(t,A,u,l):
    """Gompertz growth equation"""
    y = A * np.exp(-np.exp(((u*np.exp(1)/A)*(l-t))+1))
    return y

def log_linear(x, a, b):
    """Linear logistic regression"""
    y = np.exp(a + (b*x));
    return y

def r_squared(y,y_approx):
    """Calculates the coefficent of determination"""
    residuals = y - y_approx;
    ss_res = np.sum(residuals**2);
    ss_tot = np.sum((y-np.mean(y))**2);
    return 1.0 - (ss_res / ss_tot)

### 1.2.1
##### Above, I've included the gompertz function, yet there are a few other growth models in Zwietering et al 1990. Can you write your own grwoth model function or one of those included in Zwietering et al 1990?

# 2. Setting variables and parameters

#### For the gompertz growth model below we will display the results and curves for various growth paramaters.

## 2.1 Generate an array of time values 
#### Growth data is usually collected as some function of time. Don't forget units may matter! Here we are defining an array of time points in hours.

In [ ]:
time = np.arange(0,42); ## What does np.arange do?
print time[:5] ## Print the first 5 time points
print len(time) ## make sure there are 42 values (hours)

### 2.1.1
##### How could we change the code above to get 72 (hours) time points?
### 2.1.2
##### How could we change the code above to convert the units of our time points from hours to minutes?

## 2.2 Set parameters of growth model

#### Recall that our growth models need three parameters, A, u and l. 
### 2.2.1
##### What do these parameters, A, u, and l represent?

In [ ]:
A = [0.25, 0.5, 0.75] ## Set 3 different values for the carrying capacity
u = [0.01, 0.05, 0.09] ## ... the max growth rate
l = [20, 10, 5.0] ## ... and the lag (in hours)

###### If we changed our units of time above from hours (to say minutes) be sure to change the units of lag. 

# 3. Calculate values from Growth (Gompertz) model

#### Below we will feed into the gompertz growth function our values of A, u, l above. 

## 3.1 Calling the growth function

In [ ]:
Y1 = gompertz(time,A[0],u[0],l[0]); ## Here we feed into our gompertz function the values of A, u, and l.
Y2 = gompertz(time,A[1],u[1],l[1]);
Y3 = gompertz(time,A[2],u[2],l[2]); ## What do these values return to you?

### 3.1.1
##### For our other growth models how would we call them as we did above for the gompertz growth functions?

## 3.2 Display our Gompertz growth curves

### Make some figures! 

In [ ]:
plt.plot(time,Y1,'red'); ## Plot models
plt.plot(time,Y2,'orange');
plt.plot(time,Y3,'olive');
plt.ylim(-0.1,1); ## Set y limits
plt.title('Gompertz Growth Models',fontsize=18); ## add title and labels
plt.xlabel('Time \n( hrs )',fontsize=14);
plt.ylabel('Growth \n( a.u. )',fontsize=14);

### You can set your own parameters for growth equations in the cells above. 
### Play around with these models and parameters and make various figures. 
### 3.2.1
##### What happens when we have longer or shorter lag times but the same values for A (carrygin capacity) and u (max growth rate)? 
### 3.2.2
##### What happens when we have faster growth rates but similar l (lag) and A (carrying capacity)? 
### 3.2.3
##### If A, the carrying capacity, is large, is it possible to reach this value if we have only have 48 hours?
### 3.2.4
##### What do these curves look like for the logistic equation and log-linear regression, with and without similar values for A, u, and l?

# 4 Data Importation & Manipulation

## 4.1 Bring in the data with pandas

### Here we are bringing in "test_data" from a previous class.
### pandas takes in a path to the data as input (You will need to download this data). You may need to change this path for your computer

In [ ]:
plate = pd.read_excel('test_data.xlsx',skiprows=1) 

## 4.2 See first few (5) rows of data

### Use the pandas function .head() to see the first 5 rows of data

In [ ]:
plate.head()

## 4.3 Transpose functions
#### Much like the .head() function, other functions on pandas dataframes such as the transpose (.T) can be used

In [ ]:
plate.T

### 4.3.1
##### What column (or row) in the dataframe above contains the time points, what units are these time points in, and how many are there?
### 4.3.2
##### What information does the function len() give you? Try typing in ?len() in a cell
### 4.3.3
##### Using the len() or .shape() functions how many test samples (not time points) are in the data?

# 5 Analysis of test data

## 5.1 Baseline data
### The first time point is often subtracted to account for the differences in the initial inoculation. We can correct for the initial culture size by subtracting this value.

In [ ]:
sample = 'test1' ## Set the sample for analysis
ydata = plate[sample].values ## Take the sample values out of the dataframe and convert to an array
ydata = ydata - ydata[0] ## Subtract the initial values 
xdata = plate['(hrs)'].values ## Collect time points

## 5.2 Utilize the curve_fit function to fit growth models to our data

### 5.2.1
##### What does curve_fit do (you can check by typing in ?curve_fit in a cell)?

#### The function curve_fit is not perfect. It can have trouble dealing with small values like ours. To avoid overflow errors we can scale our values between zero and one by normilizing via the maximum and re-scaling our parameters after fitting curves.

In [ ]:
scale = max(ydata) ## Collect the max y-value for this sample
popt_gom, pcov_gom = curve_fit(gompertz, xdata, ydata/scale); ## Here we are deviding the y-values by the maximum
popt_gom[:2] = popt_gom[:2] * scale ## We need to rescale the carrying capacity and maximum growth rate (but not lag)
print popt_gom

## 5.3 Calculate the approximation from our curve fitting

In [ ]:
yhat_gom = gompertz(xdata,*popt_gom) ## here we are putting into gompertz the values from popt_gom

### 5.3.1
##### What does the "*" symbol do to the variable popt_gom?

## 5.4 Calculate the coefficient of determenation

In [ ]:
print "R2 Gompertz:", round(r_squared(ydata,yhat_gom),3)

## 5.5 Plot our data and fit

In [ ]:
plt.plot(xdata, ydata, '.k', label='Data',alpha=.5); ## Plot sample data
plt.plot(xdata, yhat_gom,'-g', label='Gompertz',alpha=.9); ## Plot growth model
plt.annotate("u: " + str(round(popt_gom[1],3)),(20,.15)); ## Add in text with the calculated parameters (u)
plt.annotate("A: " + str(round(popt_gom[0],3)),(xdata[-10],popt_gom[0]+.01)); ## ... for A
plt.annotate("l: " + str(round(popt_gom[-1],3)),(popt_gom[-1],0)); ## ... for l 
plt.ylabel('Optical Density\n( a.u. )',fontsize=14); ## Add titles, legends, and labels 
plt.xlabel('Time \n( hrs )',fontsize=14); ## Add x axis label
plt.title('Gompertz Fit & Test Data',fontsize=16); ## Add y axis label
plt.legend(loc='lower right'); ## Place the legend in lower right corner
plt.tight_layout(); ## This makes everything pretty
plt.savefig('Fit_to_growth.jpeg'); ## This will save the figure

### 5.5.1
##### What about the other samples in the data set; can you run our analysis and plot more than one sample on a figure or sub-figures? 
Hint: check out plt.subplots()
### 5.5.2
##### Explore the other models introduced in class and written above, for a given sample do we see differences in the estimated parameters from different models?
### 5.5.3
##### From the question above, for a given sample, plot on the same figure the gompertz and logistic growth models.